# Imports

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression,Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

import pickle


import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV



from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout, LeakyReLU
from sklearn.preprocessing import StandardScaler
import tensorflow.keras.backend as K
from tensorflow import keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import matplotlib.patches as mpatches
import tensorflow as tf


import plotly.express as px
import plotly.graph_objects as go

!pip install backtesting
from backtesting import Strategy
from backtesting import Backtest

# Pre-processing

In [2]:
from google.colab import files

uploaded = files.upload()

Saving combinedv1_textblob.csv to combinedv1_textblob.csv


In [3]:
data_df = pd.read_csv("combinedv1_textblob.csv")

# Drop uneeded columns for model
data_df = data_df.set_index('Date')
OHL_data_df = data_df[['Open', 'High', 'Low']]
data_df.drop(columns = ["Unnamed: 0", "Unnamed: 0.1", "Open", "High", "Low", "Adj Close","RSI","Volume","subjectivity","polarity"], inplace = True)

# Shift close prices (labels) back by 1 day (e.g. suppose today 20th, we want to predict for 21st)
data_df['Close'] = data_df['Close'].shift(-1)
data_df.dropna(inplace = True)

In [4]:
data_df.head()

,Close,SMA14,On Balance Volume
Date,,,
2016-08-21,586.752991,581.039010,-234433596.0
2016-08-22,583.414978,580.731794,-161589596.0
2016-08-23,580.182007,580.418505,-246938796.0
2016-08-24,577.760986,579.567003,-303266996.0
2016-08-25,579.651001,578.755646,-439396996.0


## Train Test Splits
- y_labels: Close prices
- X_features: Features (Vol, SMA, RSI, OBV, Subjectivity, Polarity)

In [5]:
# Split labels and features
y_labels = data_df["Close"]
X_features = data_df.copy().drop(columns = ["Close"])

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.20, shuffle = False)


In [6]:
# Plot data
training_split = pd.merge(X_train, y_train, left_index = True, right_index = True)
test_split = pd.merge(X_test, y_test, left_index = True, right_index = True)

# Models


## KOK WEE's modified SVM

In [10]:
# Model
svr_poly_model = SVR(kernel='poly', C=100, degree=2)
svr_poly_model.fit(X_train, y_train)

svr_poly_confidence = svr_poly_model.score(X_test, y_test)
print("svr_poly_confidence: ", svr_poly_confidence)

svr_poly_results_df = X_test.copy()
svr_poly_results_df['results'] = svr_poly_model.predict(X_test)


# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(y = training_split['Close'], x = training_split.index, name = "Train Data"))
fig.add_trace(go.Scatter(y = svr_poly_results_df['results'], x = svr_poly_results_df.index, name = "Results"))
fig.add_trace(go.Scatter(y = test_split['Close'], x = test_split.index, name = "Test Data"))
fig.show()

svr_poly_confidence:  0.6789991489315811


In [19]:
print(np.sqrt(mean_squared_error(y_test, svr_poly_results_df['results'])))

9360.363357818778


In [ ]:
# Backtest Model (SVR)
svr_poly_model = SVR(kernel='poly', C=10, degree=2)
svr_poly_model.fit(X_train, y_train)
svr_poly_results = svr_poly_model.predict(X_test)

svr_poly_confidence = svr_poly_model.score(X_test, y_test)
print("svr_poly_confidence: ", svr_poly_confidence)

# Results Data (20% Original DF + results from model)
results_data = test_split.copy()
results_data['Results'] = svr_poly_results

# Backtest Data (Results Data + inter-day % difference)
backtest_data = results_data.copy().merge(OHL_data_df, left_index = True, right_index = True)
backtest_data['ResultsDiff'] = backtest_data['Results'].diff()
backtest_data.dropna(inplace = True)
backtest_data['ResultsDiffPerc'] = backtest_data['ResultsDiff'] / backtest_data['Results']
backtest_data.set_index(pd.DatetimeIndex(backtest_data.index), inplace = True)

# backtest_data.head()

svr_poly_confidence:  0.701440499248366


In [ ]:
# Backtesting.py
def getCol(data_df, colName):
  return pd.Series(data_df[colName])

class ModelBacktest(Strategy):

    def init(self):
        # Precompute the two moving averages
        self.resultsDiffPerc = self.I(getCol, self.data, "ResultsDiffPerc", name = "Predicted Perc Difference")

    def next(self):
        tradeThreshold = 0.010 # test
        diff = self.resultsDiffPerc
        
        if diff > tradeThreshold:
            # if our prediction is more than threshold we close existing positions
            self.position.close()
        
        elif diff < tradeThreshold:
            # if our prediction is more than threshold we buy BTC
            self.buy()

bt = Backtest(backtest_data, ModelBacktest, trade_on_close = True, cash = 100000)
# Trade on close because we consolidate news to make a prediction (end of day)

stats = bt.run()
stats

Start                     2020-07-30 00:00:00
End                       2021-08-06 00:00:00
Duration                    372 days 00:00:00
Exposure Time [%]                     72.3944
Equity Final [$]                       133051
Equity Peak [$]                        193503
Return [%]                            33.0509
Buy & Hold Return [%]                 284.297
Return (Ann.) [%]                     34.1255
Volatility (Ann.) [%]                 69.5931
Sharpe Ratio                         0.490357
Sortino Ratio                         1.00549
Calmar Ratio                         0.816153
Max. Drawdown [%]                    -41.8127
Avg. Drawdown [%]                     -7.8221
Max. Drawdown Duration      116 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                   98
Win Rate [%]                          57.1429
Best Trade [%]                        15.5186
Worst Trade [%]                       -19.699
Avg. Trade [%]                    

In [ ]:
bt.plot()

Row(id='1404', ...)

## LINEAR REGRESSION (JOLIE)

In [21]:
linear = LinearRegression()
model = linear.fit(X_train,y_train)
predict = model.score(X_test, y_test)


linear_results_df = X_test.copy()
linear_results_df['results'] = model.predict(X_test)

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(y = training_split['Close'], x = training_split.index, name = "Train Data"))
fig.add_trace(go.Scatter(y = linear_results_df['results'], x = linear_results_df.index, name = "Results"))
fig.add_trace(go.Scatter(y = test_split['Close'], x = test_split.index, name = "Test Data"))
fig.show()

In [22]:
print(np.sqrt(mean_squared_error(y_test, linear_results_df['results'])))

3796.5641490111134


In [ ]:
linear_results_df = X_test.copy()
linear_results_df['results'] = model.predict(X_test)

# Results Data (20% Original DF + results from model)
results_data = test_split.copy()
results_data['Results'] = model.predict(X_test)

# Backtest Data (Results Data + inter-day % difference)
backtest_data = results_data.copy().merge(OHL_data_df, left_index = True, right_index = True)
backtest_data['ResultsDiff'] = backtest_data['Results'].diff()
backtest_data.dropna(inplace = True)
backtest_data['ResultsDiffPerc'] = backtest_data['ResultsDiff'] / backtest_data['Results']
backtest_data.set_index(pd.DatetimeIndex(backtest_data.index), inplace = True)

# backtest_data.head()

In [ ]:
# Backtesting.py
def getCol(data_df, colName):
  return pd.Series(data_df[colName])

class ModelBacktest(Strategy):

    def init(self):
        # Precompute the two moving averages
        self.resultsDiffPerc = self.I(getCol, self.data, "ResultsDiffPerc", name = "Predicted Perc Difference")

    def next(self):
        tradeThreshold = 0.010 # test
        diff = self.resultsDiffPerc
        
        if diff > tradeThreshold:
            # if our prediction is more than threshold we close existing positions
            self.position.close()
        
        elif diff < tradeThreshold:
            # if our prediction is more than threshold we buy BTC
            self.buy()

bt = Backtest(backtest_data, ModelBacktest, trade_on_close = True, cash = 100000)
# Trade on close because we consolidate news to make a prediction (end of day)

stats = bt.run()
stats

Start                     2020-07-30 00:00:00
End                       2021-08-06 00:00:00
Duration                    372 days 00:00:00
Exposure Time [%]                     74.6479
Equity Final [$]                       160820
Equity Peak [$]                        235914
Return [%]                            60.8195
Buy & Hold Return [%]                 284.297
Return (Ann.) [%]                     62.9863
Volatility (Ann.) [%]                 114.583
Sharpe Ratio                           0.5497
Sortino Ratio                         1.53064
Calmar Ratio                          1.28265
Max. Drawdown [%]                    -49.1062
Avg. Drawdown [%]                    -8.39893
Max. Drawdown Duration      116 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                                   15
Win Rate [%]                          73.3333
Best Trade [%]                        20.5034
Worst Trade [%]                      -29.4662
Avg. Trade [%]                    

In [ ]:
bt.plot()

Row(id='4899', ...)

## Stacking Regressor

In [23]:
estimators = [
        ('RFR', RandomForestRegressor()),
        ('svr', SVR(kernel='poly')),
        ('linear',LinearRegression()),
        ('lasso', Lasso())
    ]
reg = StackingRegressor(
     estimators=estimators,
     final_estimator=Lasso()
 )

In [24]:
model = reg.fit(X_train, y_train)
predict = model.score(X_test, y_test)

stack_results_df = X_test.copy()
stack_results_df['results'] = model.predict(X_test)

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(y = training_split['Close'], x = training_split.index, name = "Train Data"))
fig.add_trace(go.Scatter(y = stack_results_df['results'], x = stack_results_df.index, name = "Results"))
fig.add_trace(go.Scatter(y = test_split['Close'], x = test_split.index, name = "Test Data"))
fig.show()

In [25]:
print(np.sqrt(mean_squared_error(y_test, stack_results_df['results'])))

6343.526458347222


In [ ]:
estimators = [
        ('RFR', RandomForestRegressor()),
        ('svr', SVR(kernel='poly')),
        ('linear',LinearRegression()),
        ('lasso', Lasso(normalize=True, tol=1e-2))
    ]
reg = StackingRegressor(
     estimators=estimators,
     final_estimator=Lasso(normalize=True, tol=1e-2)
 )

model = reg.fit(X_train, y_train)
predict = model.score(X_test, y_test)

stack_results_df = X_test.copy()
stack_results_df['results'] = model.predict(X_test)

# Results Data (20% Original DF + results from model)
results_data = test_split.copy()
results_data['Results'] = model.predict(X_test)

# Backtest Data (Results Data + inter-day % difference)
backtest_data = results_data.copy().merge(OHL_data_df, left_index = True, right_index = True)
backtest_data['ResultsDiff'] = backtest_data['Results'].diff()
backtest_data.dropna(inplace = True)
backtest_data['ResultsDiffPerc'] = backtest_data['ResultsDiff'] / backtest_data['Results']
backtest_data.set_index(pd.DatetimeIndex(backtest_data.index), inplace = True)

# backtest_data.head()

In [ ]:
# Backtesting.py
def getCol(data_df, colName):
  return pd.Series(data_df[colName])

class ModelBacktest(Strategy):

    def init(self):
        # Precompute the two moving averages
        self.resultsDiffPerc = self.I(getCol, self.data, "ResultsDiffPerc", name = "Predicted Perc Difference")

    def next(self):
        tradeThreshold = 0.010 # test
        diff = self.resultsDiffPerc
        
        if diff > tradeThreshold:
            # if our prediction is more than threshold we close existing positions
            self.position.close()
        
        elif diff < tradeThreshold:
            # if our prediction is more than threshold we buy BTC
            self.buy()

bt = Backtest(backtest_data, ModelBacktest, trade_on_close = True, cash = 100000)
# Trade on close because we consolidate news to make a prediction (end of day)

stats = bt.run()
stats

Start                     2020-07-30 00:00:00
End                       2021-08-06 00:00:00
Duration                    372 days 00:00:00
Exposure Time [%]                     74.6479
Equity Final [$]                       160820
Equity Peak [$]                        235914
Return [%]                            60.8195
Buy & Hold Return [%]                 284.297
Return (Ann.) [%]                     62.9863
Volatility (Ann.) [%]                 114.583
Sharpe Ratio                           0.5497
Sortino Ratio                         1.53064
Calmar Ratio                          1.28265
Max. Drawdown [%]                    -49.1062
Avg. Drawdown [%]                    -8.39893
Max. Drawdown Duration      116 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                                   15
Win Rate [%]                          73.3333
Best Trade [%]                        20.5034
Worst Trade [%]                      -29.4662
Avg. Trade [%]                    

In [ ]:
bt.plot()

Row(id='2802', ...)

## XGBoosting

In [26]:
%%time

parameters = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
}
# xgb.set_config(verbosity=0)

args = {'eval_set': [(X_test, y_test)]}


model = xgb.XGBRegressor()
# model.set_params(**args)
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)

print(f'Best params: {clf.best_params_}')
print(f'Best validation score = {clf.best_score_}')

[12:02:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [28]:
%%time
model = xgb.XGBRegressor(**clf.best_params_, objective='reg:squarederror').fit(X_train, y_train, verbose=False)

CPU times: user 53.1 ms, sys: 940 µs, total: 54 ms
Wall time: 54.1 ms


In [29]:
boost_results_df = X_test.copy()
boost_results_df['results'] = model.predict(X_test)

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(y = training_split['Close'], x = training_split.index, name = "Train Data"))
fig.add_trace(go.Scatter(y = boost_results_df['results'], x = boost_results_df.index, name = "Results"))
fig.add_trace(go.Scatter(y = test_split['Close'], x = test_split.index, name = "Test Data"))
fig.show()

In [30]:
print(np.sqrt(mean_squared_error(y_test, boost_results_df['results'])))

23216.52235374443


In [ ]:
# Results Data (20% Original DF + results from model)
results_data = test_split.copy()
results_data['Results'] = model.predict(X_test)

# Backtest Data (Results Data + inter-day % difference)
backtest_data = results_data.copy().merge(OHL_data_df, left_index = True, right_index = True)
backtest_data['ResultsDiff'] = backtest_data['Results'].diff()
backtest_data.dropna(inplace = True)
backtest_data['ResultsDiffPerc'] = backtest_data['ResultsDiff'] / backtest_data['Results']
backtest_data.set_index(pd.DatetimeIndex(backtest_data.index), inplace = True)

# backtest_data.head()

In [ ]:
# Backtesting.py
def getCol(data_df, colName):
  return pd.Series(data_df[colName])

class ModelBacktest(Strategy):

    def init(self):
        # Precompute the two moving averages
        self.resultsDiffPerc = self.I(getCol, self.data, "ResultsDiffPerc", name = "Predicted Perc Difference")

    def next(self):
        tradeThreshold = 0.010 # test
        diff = self.resultsDiffPerc
        
        if diff > tradeThreshold:
            # if our prediction is more than threshold we close existing positions
            self.position.close()
        
        elif diff < tradeThreshold:
            # if our prediction is more than threshold we buy BTC
            self.buy()

bt = Backtest(backtest_data, ModelBacktest, trade_on_close = True, cash = 100000)
# Trade on close because we consolidate news to make a prediction (end of day)

stats = bt.run()
stats

Start                     2020-07-30 00:00:00
End                       2021-08-06 00:00:00
Duration                    372 days 00:00:00
Exposure Time [%]                     96.9014
Equity Final [$]                       357494
Equity Peak [$]                        517395
Return [%]                            257.494
Buy & Hold Return [%]                 284.297
Return (Ann.) [%]                     270.556
Volatility (Ann.) [%]                 338.285
Sharpe Ratio                         0.799786
Sortino Ratio                         5.62569
Calmar Ratio                           5.1929
Max. Drawdown [%]                    -52.1011
Avg. Drawdown [%]                    -8.02711
Max. Drawdown Duration      116 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                   14
Win Rate [%]                          71.4286
Best Trade [%]                        140.488
Worst Trade [%]                      -12.8925
Avg. Trade [%]                    

In [ ]:
bt.plot()

Row(id='3501', ...)

## Combined model

In [32]:
# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(y = training_split['Close'], x = training_split.index, name = "Train Data"))
fig.add_trace(go.Scatter(y = combined_model, x = boost_results_df.index, name = "Results"))
fig.add_trace(go.Scatter(y = test_split['Close'], x = test_split.index, name = "Test Data"))
fig.show()

In [34]:
print(np.sqrt(mean_squared_error(y_test, combined_model)))

5564.683031743773


In [33]:
# Results Data (20% Original DF + results from model)
results_data = test_split.copy()
results_data['Results'] = combined_model

# Backtest Data (Results Data + inter-day % difference)
backtest_data = results_data.copy().merge(OHL_data_df, left_index = True, right_index = True)
backtest_data['ResultsDiff'] = backtest_data['Results'].diff()
backtest_data.dropna(inplace = True)
backtest_data['ResultsDiffPerc'] = backtest_data['ResultsDiff'] / backtest_data['Results']
backtest_data.set_index(pd.DatetimeIndex(backtest_data.index), inplace = True)

In [ ]:
# Backtesting.py
def getCol(data_df, colName):
  return pd.Series(data_df[colName])

class ModelBacktest(Strategy):

    def init(self):
        # Precompute the two moving averages
        self.resultsDiffPerc = self.I(getCol, self.data, "ResultsDiffPerc", name = "Predicted Perc Difference")

    def next(self):
        tradeThreshold = 0.010 # test
        diff = self.resultsDiffPerc
        
        if diff > tradeThreshold:
            # if our prediction is more than threshold we close existing positions
            self.position.close()
        
        elif diff < tradeThreshold:
            # if our prediction is more than threshold we buy BTC
            self.buy()

bt = Backtest(backtest_data, ModelBacktest, trade_on_close = True, cash = 100000)
# Trade on close because we consolidate news to make a prediction (end of day)

stats = bt.run()
stats

Start                     2020-07-30 00:00:00
End                       2021-08-06 00:00:00
Duration                    372 days 00:00:00
Exposure Time [%]                     79.4366
Equity Final [$]                       164040
Equity Peak [$]                        258845
Return [%]                            64.0397
Buy & Hold Return [%]                 284.297
Return (Ann.) [%]                     66.3427
Volatility (Ann.) [%]                 102.188
Sharpe Ratio                         0.649223
Sortino Ratio                         1.69893
Calmar Ratio                          1.25546
Max. Drawdown [%]                    -52.8433
Avg. Drawdown [%]                    -7.15313
Max. Drawdown Duration      116 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                   72
Win Rate [%]                             62.5
Best Trade [%]                        15.0897
Worst Trade [%]                      -21.5089
Avg. Trade [%]                    

In [ ]:
bt.plot()

Row(id='5598', ...)